Al final de este paso, comprenderá los conceptos de subajuste y sobreajuste y podrá aplicar estas ideas para hacer que sus modelos sean más precisos.

## Experimentando con diferentes modelos

Ahora que dispone de una forma fiable de medir la precisión del modelo, puede experimentar con modelos alternativos y ver cuál ofrece las mejores predicciones. ¿Pero qué alternativas tienes para los modelos?

Puede ver en la documentación de scikit-learn que el modelo de árbol de decisión tiene muchas opciones (más de las que querrá o necesitará durante mucho tiempo). Las opciones más importantes determinan la profundidad del árbol. Recuerde de la primera lección de este curso que la profundidad de un árbol es una medida de cuántas divisiones realiza antes de llegar a una predicción. Este es un árbol relativamente poco profundo.

En la práctica, no es raro que un árbol tenga 10 divisiones entre el nivel superior (todas las casas) y una hoja. A medida que el árbol se hace más profundo, el conjunto de datos se divide en hojas con menos casas. Si un árbol solo tuvo 1 división, divide los datos en 2 grupos. Si cada grupo se vuelve a dividir, obtendríamos 4 grupos de casas. Dividir cada uno de ellos nuevamente crearía 8 grupos. Si seguimos duplicando el número de grupos añadiendo más divisiones en cada nivel, tendremos 210
   grupos de casas cuando lleguemos al décimo nivel. Son 1024 hojas.

Cuando dividimos las casas entre muchas hojas, también tenemos menos casas en cada hoja. Las hojas con muy pocas casas harán predicciones bastante cercanas a los valores reales de esas casas, pero pueden hacer predicciones muy poco confiables para datos nuevos (porque cada predicción se basa solo en unas pocas casas).

Este es un fenómeno llamado sobreajuste, donde un modelo coincide casi perfectamente con los datos de entrenamiento, pero obtiene malos resultados en la validación y otros datos nuevos. Por otro lado, si hacemos que nuestro árbol sea muy poco profundo, no dividirá las casas en grupos muy distintos.

En un extremo, si un árbol divide las casas en sólo 2 o 4, cada grupo todavía tiene una amplia variedad de casas. Las predicciones resultantes pueden estar lejanas para la mayoría de las casas, incluso en los datos de entrenamiento (y también serán malas en la validación por la misma razón). Cuando un modelo no logra capturar distinciones y patrones importantes en los datos, por lo que funciona mal incluso en datos de entrenamiento, eso se denomina subajuste.

Dado que nos preocupamos por la precisión de los datos nuevos, que estimamos a partir de nuestros datos de validación, queremos encontrar el punto óptimo entre el desajuste y el sobreajuste.  

### Ejemplo
Existen algunas alternativas para controlar la profundidad del árbol y muchas permiten que algunas rutas a través del árbol tengan mayor profundidad que otras. Pero el argumento max_leaf_nodes proporciona una forma muy sensata de controlar el sobreajuste frente al desajuste. Cuantas más hojas permitimos que haga el modelo, más nos movemos del área de subajuste en el gráfico anterior al área de sobreajuste.

Podemos usar una función de utilidad para ayudar a comparar puntuaciones MAE de diferentes valores para max_leaf_nodes:

In [1]:
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor

def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)

Los datos se cargan en train_X, val_X, train_y y val_y usando el código que ya has visto (y que ya has escrito).

In [2]:
# Data Loading Code Runs At This Point
import pandas as pd
    
# Load data
melbourne_file_path = '/kaggle/input/melbourne-housing-snapshot/melb_data.csv'
melbourne_data = pd.read_csv(melbourne_file_path) 
# Filter rows with missing values
filtered_melbourne_data = melbourne_data.dropna(axis=0)
# Choose target and features
y = filtered_melbourne_data.Price
melbourne_features = ['Rooms', 'Bathroom', 'Landsize', 'BuildingArea', 
                        'YearBuilt', 'Lattitude', 'Longtitude']
X = filtered_melbourne_data[melbourne_features]

from sklearn.model_selection import train_test_split

# split data into training and validation data, for both features and target
train_X, val_X, train_y, val_y = train_test_split(X, y,random_state = 0)

Podemos usar un bucle for para comparar la precisión de modelos creados con diferentes valores para max_leaf_nodes.

In [4]:
# compare MAE with differing values of max_leaf_nodes
for max_leaf_nodes in [5, 50, 490, 500, 510, 5000]:
    my_mae = get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y)
    print("Max leaf nodes: %d  \t\t Mean Absolute Error:  %d" %(max_leaf_nodes, my_mae))

Max leaf nodes: 5  		 Mean Absolute Error:  347380
Max leaf nodes: 50  		 Mean Absolute Error:  258171
Max leaf nodes: 490  		 Mean Absolute Error:  243530
Max leaf nodes: 500  		 Mean Absolute Error:  243495
Max leaf nodes: 510  		 Mean Absolute Error:  244352
Max leaf nodes: 5000  		 Mean Absolute Error:  255575


### Conclusión
Aquí está la conclusión: los modelos pueden sufrir cualquiera de los siguientes problemas:

Sobreajuste: capturar patrones espurios que no se repetirán en el futuro, lo que lleva a predicciones menos precisas, o

Desajuste: no capturar patrones relevantes, lo que nuevamente conduce a predicciones menos precisas.
Usamos datos de validación, que no se usan en el entrenamiento de modelos, para medir la precisión de un modelo candidato. Esto nos permite probar muchos modelos candidatos y quedarnos con el mejor.